In [35]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datetime
import xgboost
import sklearn
import sklearn.ensemble as ensemble
#from imblearn.ensemble import BalancedRandomForestClassifier


from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from xgboost import XGBRFClassifier

from sklearn.feature_selection import RFE
from platform import python_version


In [36]:
# costumized stuff  
patient_data = False
Simple =  True
print('The scikit-learn version is {}.'.format(sklearn.__version__))


The scikit-learn version is 1.0.2.


In [37]:
cols_to_include = ['cohort','Patient','HLA_allele','Mut_peptide','response','Partition',
                   'Aro','mw','pI', 'Inst', 'CysRed','RankEL','RankBA','NetMHCExp',
                        'Expression','SelfSim','Prime','PropHydroAro','HydroCore',
                        'PropSmall','PropAro','PropBasic','PropAcidic','DAI','Stability','Foreigness',
                        'CelPrev','PrioScore','CYT','HLAexp','MCPmean']#

In [38]:
# run the data with or without the patient data for tumor microenviroment and with or withput selected cols

if patient_data == True and Simple == False: 
    cols_to_drop = ['cohort','Patient','HLA_allele','Mut_peptide','response','Partition',] 
    Model = "TME_included"
if patient_data == False and Simple == False: 
    cols_to_drop = ['cohort','Patient','HLA_allele','Mut_peptide','response','Partition',
                   'CYT','HLAexp','MCPmean']
    Model = "TME_excluded"
if patient_data == False and Simple == True: 
    cols_to_drop = ['cohort','Patient','HLA_allele','Mut_peptide','response','Partition',
                   'CelPrev','PrioScore','CYT','HLAexp','MCPmean'] 
    Model = "Simple"


In [39]:

data = pd.read_csv('../../data/03_data_for_CV/IMPROVE/03_3_final_peptide_features_Partition_for_CV.txt', sep='\t')
Var_importance_filename = '../../results/5_fold_CV/'+Model+'/Feature_importance_wo_prime' + Model +'.txt'


In [40]:
print(Model)
print(data.columns)

Simple
Index(['Sample', 'Patient', 'HLA_allele', 'Norm_peptide', 'Mut_peptide',
       'RankEL', 'RankBA', 'RankEL_wt', 'Gene_ID', 'Transcript_ID',
       'Genomic_Position', 'Protein_position', 'Mutation_Consequence',
       'VarAlFreq', 'Gene_Symbol', 'Cancer_Driver_Gene', 'Expression',
       'PrioScore', 'CelPrev', 'cohort', 'identity', 'response', 'validation',
       'PeptNorm', 'PeptLen', 'Core', 'Of', 'Gp', 'Gl', 'Ip', 'Il',
       'RankEL_4.1', 'RankBA_4.1', 'RankEL_wt_4.1', 'Stability', 'Prime',
       'DAI_4.1', 'CoreNonAnchor', 'Loci', 'HydroAll', 'HydroCore',
       'PropSmall', 'PropAro', 'PropBasic', 'PropAcidic', 'SelfSim', 'mw',
       'Aro', 'Inst', 'PropHydroAro', 'CysRed', 'pI', 'DAI',
       'Misense_mutation', 'Frameshift_mutation', 'Inframe_deletion_mutation',
       'Inframe_insertion', 'IB_CB', 'IB_CB_cat', 'Foreigness', 'Sample_TME',
       'Tcells', 'TcellsCD8', 'CytoxLympho', 'Blinage', 'NKcells', 'Monocytes',
       'MyeloidDC', 'Neutrophils', 'Endothelial'

In [41]:
data = data[cols_to_include]
print(data)

        cohort  Patient  HLA_allele  Mut_peptide  response  Partition  \
0      bladder  BC-1849  HLA-A01:01    YTDQISKYA         1          0   
1      bladder  BC-1849  HLA-A01:01    QLEQLMQLY         1          0   
2      bladder  BC-1849  HLA-A01:01  ILEYTDQISKY         1          0   
3      bladder  BC-2389  HLA-A01:01    CIDFQPDIY         1          4   
4      bladder  BC-2389  HLA-A01:01   SCIDFQPDIY         1          4   
...        ...      ...         ...          ...       ...        ...   
17515   Basket    RH-08  HLA-B44:02  AEASAHSPRSY         0          3   
17516   Basket    RH-08  HLA-A02:01    VLLSAETMA         0          3   
17517   Basket    RH-08  HLA-B44:02   AETMATTLLA         0          3   
17518   Basket    RH-08  HLA-B44:02    AETMATTLL         0          3   
17519   Basket    RH-08  HLA-B44:02   SAETMATTLL         0          3   

            Aro         mw        pI       Inst  ...  PropBasic  PropAcidic  \
0      0.222222  1088.1673  6.324010  25.766

In [42]:
# nested cross validation
outfile = open(Var_importance_filename ,'w')
n_sample = 50
partitioning_list = data.Partition.unique()
#partitioning_list.sort()
print(partitioning_list)
pred_df = pd.DataFrame()
for i in partitioning_list:
    av_pred_rf = 0
    print(i)
    for n in range(n_sample):  
        test = data[data.Partition == i]
        train = data[data.Partition != i]
        info = test[['cohort',"Mut_peptide", "HLA_allele","Patient","Partition"]]
        train_pos = train[train.response==1]
        train_neg = train[train.response==0].sample(n=500, random_state=n) 
        train = shuffle(pd.concat([train_pos, train_neg], axis=0), random_state = 42).reset_index(drop=True)
        train = train[~train.Mut_peptide.isin(test["Mut_peptide"])]
        X_train = train.drop(cols_to_drop, axis=1).reset_index(drop=True)
        
        y_train = train["response"]
        feature_list = list(X_train.columns)
        


        # Instantiate the RF and the MLP # 
        rf = RandomForestClassifier(random_state = 42, max_depth = 6, min_samples_leaf = 6,
                                    n_estimators = 2000, n_jobs=-1) #class_weight={0:1,1:3}

                # Train the models on training data
        rf.fit(X_train, y_train)
        
        # pickle dumb model 
        pickle_model = '../../models/'+ Model +'/rf' + str(i) + '_' + Model +'_' + str(n) + '.pkl'
        pickle.dump(rf, open(pickle_model, 'wb'))
        
        X_test = test.drop(cols_to_drop, axis=1).reset_index(drop=True)
        y_test = test["response"]  
        len(X_test)

        #Get fetures importance per partition
        importances = list(rf.feature_importances_)
        feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]
        # Sort
        feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)
     #   [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances]
      #  gather all feature importance 
        for pair in feature_importances:
            outfile.write(str(pair[0]) + "\t" + str(pair[1]) + "\t" + str(i) + "\n")

        prediction_rf = rf.predict_proba(X_test)

        
        av_pred_rf += prediction_rf
    
    pred_val_rf = av_pred_rf/n_sample
        
    pred_val_rf = pd.DataFrame(pred_val_rf[:,1])
    pred_val_rf.reset_index(drop=True, inplace=True)
    mapping = {pred_val_rf.columns[0]: "prediction_rf"}
    pred_val_rf = pred_val_rf.rename(columns=mapping)

    X_test.reset_index(drop=True, inplace=True)
    y_test.reset_index(drop=True, inplace=True)
    info.reset_index(drop=True, inplace=True)
    print(len(X_test))
    ped_df = pd.concat([info,X_test,y_test,pred_val_rf],axis = 1)
    ped_df["Partition"] = i
    pred_df = pred_df.append(ped_df)
    print(X_train.columns)

        
#print("RF AUC:",round(roc_auc_score(pred_df.response, pred_df.prediction_rf),4))    
print("n sample:",n_sample,"RF AUC:",round(roc_auc_score(pred_df.response, pred_df.prediction_rf),4))  
outfile.close()  


[0 4 3 2 1]
0
3209
Index(['Aro', 'mw', 'pI', 'Inst', 'CysRed', 'RankEL', 'RankBA', 'NetMHCExp',
       'Expression', 'SelfSim', 'PropHydroAro', 'HydroCore', 'PropSmall',
       'PropAro', 'PropBasic', 'PropAcidic', 'DAI', 'Stability', 'Foreigness'],
      dtype='object')
4
2935
Index(['Aro', 'mw', 'pI', 'Inst', 'CysRed', 'RankEL', 'RankBA', 'NetMHCExp',
       'Expression', 'SelfSim', 'PropHydroAro', 'HydroCore', 'PropSmall',
       'PropAro', 'PropBasic', 'PropAcidic', 'DAI', 'Stability', 'Foreigness'],
      dtype='object')
3
4489
Index(['Aro', 'mw', 'pI', 'Inst', 'CysRed', 'RankEL', 'RankBA', 'NetMHCExp',
       'Expression', 'SelfSim', 'PropHydroAro', 'HydroCore', 'PropSmall',
       'PropAro', 'PropBasic', 'PropAcidic', 'DAI', 'Stability', 'Foreigness'],
      dtype='object')
2
4102
Index(['Aro', 'mw', 'pI', 'Inst', 'CysRed', 'RankEL', 'RankBA', 'NetMHCExp',
       'Expression', 'SelfSim', 'PropHydroAro', 'HydroCore', 'PropSmall',
       'PropAro', 'PropBasic', 'PropAcidic', 'DAI'

In [43]:
# print to outfile 
#currentDateTime = datetime.now().strftime("%m-%d-%Y %H-%M-%S %p")
pred_df.to_csv(r'../../results/5_fold_CV/'+Model+'/pred_df_wo_prime'+ Model +'.txt', index=None, sep=' ', mode='w')




In [34]:
#from sklearn.metrics import precision_recall_curve, auc

# Calculate precision recall curve and PR AUC score 
#precision, recall,_= precision_recall_curve(pred_df[['response']], pred_df[['prediction_rf']]) 
#pr_auc = auc(recall, precision)

# Print the PR-AUC Score 
#print("PR-AUC Score:", pr_auc)

In [35]:
#pickle_model = '../../models_modi/'+sim+'/'+TME_inc+'/rf' + str(i) + '_' + TME_inc +'_' + str(n) + '.pkl'
#print(pickle_model)


In [36]:
#pickle.load(open(pickle_model,"rb"))